# Decision Tree Model

In [1]:
import numpy as np

class Node:
  def __init__(self, feature = None, threshold = None, left = None, right = None, value = None):
    self.feature = feature
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value

  def is_leaf(self):
    return self.value is not None

class decision_tree:
  def __init__(self, min_samples_split = 2, max_depth = 100, n_feats = None):
    self.min_samples_split = min_samples_split
    self.max_depth = max_depth
    self.n_feats = n_feats

    self.root = None

  def fit(self, x, y):
    self.n_feats = x.shape[1] if not self.n_feats else min(x.shape[1] , self.n_feats)
    self.root = self._grow_tree(x, y)

  def _grow_tree(self, x, y, depth = 0):
    n_samples , n_feature = x.shape
    n_labels = len(np.unique(y))

    if (depth >= self.max_depth) or (n_labels == 1) or (n_samples < self.min_samples_split):
      leaf_value = self._most_common_label(y)
      return Node(value = leaf_value)

    features_index = np.random.choice(n_feature, self.n_feats, replace=False)
    best_feature, best_threshold = self._best_split(x, y, features_index)
    left_index , right_index = self._split(x[ : , best_feature], best_threshold)

    left_child = self._grow_tree(x[left_index, : ], y[left_index] , depth + 1)
    right_child = self._grow_tree(x[right_index, : ], y[right_index] , depth + 1)
    return Node(best_feature, best_threshold, left_child, right_child)



  def _best_split(self, x, y, features_index):
    best_gain = -1
    best_feature_index , best_threshold = None, None

    for feature_index in features_index:
      x_column = x[ : , feature_index]
      thresholds = np.unique(x_column)
      for threshold in thresholds:
        gain = self._information_gain(y, x_column, threshold)
        if gain > best_gain:
          best_gain = gain
          best_feature_index = feature_index
          best_threshold = threshold

    return best_feature_index, best_threshold

  def _information_gain(self, y,  x_column, threshold):
    entropy_parents = self._entropy(y)

    left_index, right_index = self._split(x_column, threshold)

    if len(left_index) == 0 or len(right_index) == 0:
      return 0

    n_all , n_left , n_right = len(y),  len(left_index) ,  len(right_index)
    entropy_left , entropy_right = self._entropy(y[left_index]) , self._entropy(y[right_index])

    entropy_childerns = ((n_left / n_all) * entropy_left) + ((n_right / n_all) * entropy_right)
    information_gain = entropy_parents - entropy_childerns
    return information_gain

  def _split(self, x, threshold):
    left_index = np.argwhere(x <= threshold).flatten()
    right_index = np.argwhere(x > threshold).flatten()
    return left_index, right_index

  def _entropy(self, y):
    counter = dict()
    for label in y:
      if label in counter:
        counter[label] += 1
      else:
        counter[label] = 1

    ps = np.array(list(counter.values())) / len(y)
    return -np.sum([p * np.log2(p) for p in ps if p > 0])

  def _most_common_label(self, y):
    counter = dict()
    for label in y:
      if label in counter:
        counter[label] += 1
      else:
        counter[label] = 1
    counter = dict(sorted(counter.items() , key=lambda  item : item[1]))
    most_common = list(counter.keys())[-1]
    return most_common

  def predict(self, x):
    return np.array([self._traverse_tree(point, self.root) for point in x])

  def _traverse_tree(self, x, node):
    if node.is_leaf():
      return node.value

    if x[node.feature] <= node.threshold:
      return self._traverse_tree(x, node.left)
    return self._traverse_tree(x, node.right)

  def accuracy_score(self, y_test, y_pred):
     accu = np.sum(y_test == y_pred) / len(y_test)
     return accu

# Data preparing

In [2]:
import pandas as pd
import numpy as np
import nltk

In [3]:
df = pd.read_csv("/kaggle/input/email-spam/spam.csv", encoding='latin-1')

In [4]:
df.head(3)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN


In [5]:
df.shape

(5572, 5)

In [6]:
df = df[["v1","v2"]]

In [7]:
# rename columns names
df.rename(columns={"v1":"target", "v2":"text"}, inplace=True)

In [8]:
# label encoding target column
df["target"] = df["target"].map({"ham":0, "spam":1})

In [9]:
df

,target,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [10]:
# checking null values exist in dataframe
df.isnull().sum()

target    0
text      0
dtype: int64

In [11]:
df.shape

(5572, 2)

# Text preprocessing

In [12]:
# number of characters
df["num_characters"] = df["text"].apply(len)

In [13]:
# number of words
df["num_words"] = df["text"].apply(lambda x: len(nltk.word_tokenize(x)))

In [14]:
# number of sentences
df["num_sentences"] = df["text"].apply(lambda x: len(nltk.sent_tokenize(x)))

In [15]:
df

,target,text,num_characters,num_words,num_sentences
0,0,"Go until jurong point, crazy.. Available only ...",111,23,2
1,0,Ok lar... Joking wif u oni...,29,8,2
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,155,37,2
3,0,U dun say so early hor... U c already then say...,49,13,1
4,0,"Nah I don't think he goes to usf, he lives aro...",61,15,1
...,...,...,...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...,161,35,4
5568,0,Will Ì_ b going to esplanade fr home?,37,9,1
5569,0,"Pity, * was in mood for that. So...any other s...",57,15,2
5570,0,The guy did some bitching but I acted like i'd...,125,27,1


In [16]:
from nltk.corpus import stopwords
import string
from nltk.stem.porter import PorterStemmer

In [69]:
def transform_text(text):
    # 01: transforming text into lower case
    text = text.lower()
    text = nltk.word_tokenize(text)
    
    # 02: getting alphnumeric content from text
    y = []
    for word in text:
        if word.isalnum():
            y.append(word)
    
    # 03: removing stop words and punction marks from text
    text = y[:]
    y.clear()
    for word in text:
        if word not in stopwords.words("english") and word not in string.punctuation:
            y.append(word)
            
    # 04: apply stemming 
    text = y[:]
    y.clear()
    for word in text:
        y.append(PorterStemmer().stem(word))
    
    return " ".join(y)

In [18]:
# testing the function
transform_text("ALi is goods goods how where boy's# ;$# ... >>(a)// !")

'ali good good boy'

In [19]:
df["transformed_text"] = df["text"].apply(transform_text)

In [20]:
df

,target,text,num_characters,num_words,num_sentences,transformed_text
0,0,"Go until jurong point, crazy.. Available only ...",111,23,2,go jurong point avail bugi n great world la e ...
1,0,Ok lar... Joking wif u oni...,29,8,2,ok lar joke wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,155,37,2,free entri 2 wkli comp win fa cup final tkt 21...
3,0,U dun say so early hor... U c already then say...,49,13,1,u dun say earli hor u c alreadi say
4,0,"Nah I don't think he goes to usf, he lives aro...",61,15,1,nah think goe usf live around though
...,...,...,...,...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...,161,35,4,2nd time tri 2 contact u pound prize 2 claim e...
5568,0,Will Ì_ b going to esplanade fr home?,37,9,1,b go esplanad fr home
5569,0,"Pity, * was in mood for that. So...any other s...",57,15,2,piti mood suggest
5570,0,The guy did some bitching but I acted like i'd...,125,27,1,guy bitch act like interest buy someth els nex...


In [116]:
# Xuất DataFrame sang file CSV
df.to_csv('/kaggle/working/DataEmail.csv', index=False)


# Training Model

In [21]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [22]:
# Using TF-IDF vectorizer
tf_idf = TfidfVectorizer(max_features=1000)

In [23]:
x = tf_idf.fit_transform(df["transformed_text"]).toarray()

In [24]:
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [58]:
print(x.shape)
print(df["transformed_text"])

(5572, 1000)
<class 'pandas.core.series.Series'>


In [25]:
y = df["target"].values

In [26]:
y

array([0, 0, 1, ..., 0, 0, 0])

In [27]:
# splitting data into training and testing
from sklearn.model_selection import train_test_split

In [28]:
len_train = int(0.8*len(x))
x_train = x[:len_train]
y_train = y[:len_train]
x_test = x[len_train:]
y_test = y[len_train:]

In [29]:
model = decision_tree(max_depth = 10, min_samples_split = 2)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(model.accuracy_score(y_test, y_pred))

0.9426008968609866
